# Text Summary with T5 from Huggingface Tensorflow

In [1]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.0MB 7.6MB/s 
     |████████████████████████████████| 3.2MB 46.0MB/s 
     |████████████████████████████████| 890kB 45.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=db2873feed951fcaf68a4831abcea6ac557a711bb63f361c59a429e1743c1481
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 8.4MB/s 


In [2]:
import tensorflow as tf
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration
import tensorflow_datasets as tfds
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Params

In [4]:
BATCH_SIZE = 16

SHUFFEL_SIZE = 1024

learning_rate = 3e-5

## Define Pretrained Model and Tokenizer(For Training)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = TFT5ForConditionalGeneration.from_pretrained('t5-small')

task_specific_params = model.config.task_specific_params
if task_specific_params is not None:
    model.config.update(task_specific_params.get("summarization", {}))
    
pad_token_id = tokenizer.pad_token_id

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


## Loading Fine-Tuned Model

In [5]:
tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/summarisation_tokeniser')
model = TFT5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/summarisation_model')

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/summarisation_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08, clipnorm=1.0)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

model.summary()

Model: "tf_t5for_conditional_generation"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
shared (TFSharedEmbeddings)  multiple                  16449536  
_________________________________________________________________
encoder (TFT5MainLayer)      multiple                  18881280  
_________________________________________________________________
decoder (TFT5MainLayer)      multiple                  25175808  
Total params: 60,506,624
Trainable params: 60,506,624
Non-trainable params: 0
_________________________________________________________________


## Load Dataset

In [4]:
cnn_dailymail = tfds.load(name="cnn_dailymail")

INFO:absl:No config specified, defaulting to first: cnn_dailymail/plain_text
INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: cnn_dailymail/plain_text/3.0.0
INFO:absl:Load dataset info from /tmp/tmp2wecnywutfds
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset cnn_dailymail (/root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0)


INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfTHk4NFg2SndKcjQ into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfTHk4NFg2SndKG8BdJPpt2iRo6Dpzz23CByJuAePEilB-pxbcBCHaWDs.tmp.ea98b6eb8cbe43bba6bc5008257e8f66...
INFO:absl:Downloading https://drive.google.com/uc?export=download&id=0BwmD_VLjROrfM1BxdkxVaTY2bWs into /root/tensorflow_datasets/downloads/ucexport_download_id_0BwmD_VLjROrfM1BxdkxVaTY2zVV-G71RIXPssrrvSAjt19Cy91r-9CQ2F9DMKA0uFk0.tmp.b217609769354da8be29086ae75bdd1e...
INFO:absl:Downloading https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_test.txt into /root/tensorflow_datasets/downloads/raw.gith.com_abis_cnn-dail_mast_url_list_a705_isK790OHOPsIZX-ACsObq_vchU9r5Uduh6ULX3c.txt.tmp.cca6867a877f4f6baec5f359806251ec...
INFO:absl:Downloading https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_train.txt into /root/tensorflow_datasets/downloads/raw.gith.com_abis_cnn-dail_mast_url

INFO:absl:Generating split train


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteNV8WL8/cnn_dailymail-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteNV8WL8/cnn_dailymail-train.tfrecord. Shard lengths: [17945, 17944, 17945, 17944, 17945, 17944, 17945, 17944, 17945, 17945, 17944, 17945, 17944, 17945, 17944, 17945]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteNV8WL8/cnn_dailymail-validation.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteNV8WL8/cnn_dailymail-validation.tfrecord. Shard lengths: [13368]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteNV8WL8/cnn_dailymail-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0.incompleteNV8WL8/cnn_dailymail-test.tfrecord. Shard lengths: [11490]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.SKIP.
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0


Dataset cnn_dailymail downloaded and prepared to /root/tensorflow_datasets/cnn_dailymail/plain_text/3.0.0. Subsequent calls will reuse this data.


In [6]:
train_tfds = cnn_dailymail['train']
test_tfds = cnn_dailymail['test']
val_tfds = cnn_dailymail['validation']

In [7]:
len_train = len(list(train_tfds))
len_test = len(list(test_tfds))
len_val = len(list(val_tfds))

## Loading Fine-Tuned Model

In [8]:
tokenizer = T5Tokenizer.from_pretrained('/content/drive/MyDrive/summarisation_tokeniser')
model = TFT5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/summarisation_model')

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/summarisation_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
junk
high_norm=normalize_text(highlight)
id=tokenizer.encode(high_norm, return_tensors="tf", max_length=150)
print(id)
tf.squeeze(id)

In [ ]:
d=test_ds.take(1)
for i in d:
  print(i)
test_ds

(<tf.Tensor: shape=(16, 512), dtype=int32, numpy=
array([[21603,    10,     3, ...,   221,  7721,     1],
       [21603,    10, 29329, ...,     9, 10050,     1],
       [21603,    10,     3, ...,   385,   720,     1],
       ...,
       [21603,    10,  2358, ...,     0,     0,     0],
       [21603,    10,  2177, ...,   122,  1679,     1],
       [21603,    10,  5260, ...,     3,     9,     1]], dtype=int32)>, <tf.Tensor: shape=(16, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>, <tf.Tensor: shape=(16, 150), dtype=int32, numpy=
array([[13102,     7,     7, ...,     0,     0,     0],
       [ 3719,    13, 11668, ...,     0,     0,     0],
       [ 3457,   265, 11632, ...,     0,     0,     0],
       ...,
       [ 2358,   951, 13420, ...,     0,     0,     0],
       [  774,    31,     7, ...

<PrefetchDataset shapes: ((None, 512), (None, 512), (None, 150)), types: (tf.int32, tf.int32, tf.int32)>

In [ ]:
test=test_tfds.take(2)
print(art)
print(highlight)
' '.join(pred)

a drunk teenage boy had to be rescued by security after jumping into a lions enclosure at a zoo in western india. rahul kumar, 17, clambered over the enclosure fence at the kamla nehru zoological park in ahmedabad, and began running towards the animals, shouting he would 'kill them'. mr kumar explained afterwards that he was drunk and 'thought i'd stand a good chance' against the predators. next level drunk: intoxicated rahul kumar, 17, climbed into the lions' enclosure at a zoo in ahmedabad and began running towards the animals shouting 'today i kill a lion!' mr kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security. the intoxicated teenager ran towards the lions, shouting: 'today i kill a lion or a lion kills me!' a zoo spokesman said: 'guards had earlier spotted him close to the enclosure but had no idea he was planing to enter it. 'fortunately, there are eight moats to cross before getting to where the lions usually are and he 

'English teenager rahul kumar, 17, climbed into lions enclosure at kamla nehru zoological park in ahmedabad . he ran towards the animals shouting: today i kill a lion! luckily, he fell into a moat as he could be rescued by security .'

In [ ]:
for example in test:
  article_ids, attention_mask, highlights_ids=map_func(example)
  article=example['article']
  highlight=example['highlights']

art=normalize_text(article)
high=normalize_text(highlight)
ids =tokenizer.encode_plus((model.config.prefix + art), return_tensors="tf", max_length=512) 
print(article_ids)

sum = model.generate(input_ids=ids['input_ids'], attention_mask=ids['attention_mask'])
#tokenizer("summarize: studies have shown that owning a dog is good for you hell yeah as they say ", return_tensors="tf").input_ids  # Batch size 1
pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in sum]
pred

tf.Tensor(
[21603    10     3     9 18364 21615  4940   141    12    36     3 28540
    57  1034   227 15539   139     3     9     3  7325     7 22104    44
     3     9     3   172    32    32    16  8282 18222     5     3    52
     9   107    83     3  2729  1635     6 12864     3 12818   115  3737
   147     8 22104  8227    44     8  6511   521     3    29  7392    76
     3   172    32  4478  2447    16     3     9   107  2726     9  5514
     6    11  1553  1180  1587     8  3127     6 14314    53     3    88
   133     3    31 10824   135    31     5     3    51    52     3  2729
  1635  5243 15627    24     3    88    47 18364    11     3    31 11841
    17     3    23    31    26  1518     3     9   207  1253    31   581
     8 23007     7     5   416   593 18364    10    16 14367   920     3
    52     9   107    83     3  2729  1635     6 12864     3 23626   139
     8     3  7325     7    31 22104    44     3     9     3   172    32
    32    16     3     9   107  2726    

['English teenager rahul kumar, 17, climbed into lions enclosure at kamla nehru zoological park in ahmedabad . he ran towards the animals shouting: today i kill a lion! luckily, he fell into a moat as he could be rescued by security .']

In [10]:
def normalize_text(text):
    """Lowercase and remove quotes from a TensorFlow string."""
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text,"'(.*)'", r"\1")
    return text.numpy().decode('UTF-8')

def tokenize_articles(text):
    text = normalize_text(text)
    ids = tokenizer.encode_plus((model.config.prefix + text), return_tensors="tf", max_length=512) 

    return tf.squeeze(ids['input_ids']), tf.squeeze(ids['attention_mask'])
        
def tokenize_highlights(text):
    text = normalize_text(text)
    ids = tokenizer.encode(text, return_tensors="tf", max_length=150)
    return tf.squeeze(ids)



def map_func(features):

    article_ids, attention_mask = tf.py_function(tokenize_articles, inp=[features["article"]], Tout=(tf.int32, tf.int32))
    highlights_ids = tf.py_function(tokenize_highlights, inp=[features["highlights"]], Tout=tf.int32)

    return article_ids, attention_mask, highlights_ids


In [11]:
train_ds = train_tfds.map(map_func)\
    .shuffle(SHUFFEL_SIZE)\
    .padded_batch(BATCH_SIZE, padded_shapes=([512],[512],[150]))\
    .prefetch(tf.data.experimental.AUTOTUNE)

val_ds = val_tfds.map(map_func)\
    .shuffle(SHUFFEL_SIZE)\
    .padded_batch(BATCH_SIZE, padded_shapes=([512],[512],[150]))\
    .prefetch(tf.data.experimental.AUTOTUNE)

test_ds = test_tfds.map(map_func)\
    .shuffle(SHUFFEL_SIZE)\
    .padded_batch(BATCH_SIZE, padded_shapes=([512],[512],[150]))\
    .prefetch(tf.data.experimental.AUTOTUNE)

## Define Train and Validation Step

In [12]:
@tf.function
def train_step(input_ids, input_mask, y):
    y_ids = y[:, :-1]
    lm_labels = tf.identity(y[:, 1:])
    lm_labels = tf.where(tf.equal(y[:, 1:],pad_token_id), -100, lm_labels)

    with tf.GradientTape() as tape:
        # prediction_scores: (bs, 150, 32128)
        # decoder_past_key_value_states: (bs, 512, 512), (bs, 8, 150, 64)
        # z: (bs, 512, 512)
        predictions= model(input_ids, attention_mask=input_mask, decoder_input_ids=y_ids,labels=lm_labels, training=True)
        #print(model(input_ids=input_ids, attention_mask=input_mask, decoder_input_ids=y_ids,labels=lm_labels, training=True))
        loss = loss_object(y[:, 1:],predictions.logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(y[:, 1:], predictions.logits)

In [13]:
@tf.function
def val_step(input_ids, input_mask, y):
    y_ids = y[:, :-1]
    lm_labels = tf.identity(y[:, 1:])
    lm_labels = tf.where(tf.equal(y[:, 1:],pad_token_id), -100, lm_labels)
    
    predictions= model(input_ids, attention_mask=input_mask, decoder_input_ids=y_ids,labels=lm_labels, training=False)
    v_loss = loss_object(y[:, 1:], predictions.logits)

    val_loss(v_loss)
    val_accuracy(y[:, 1:], predictions.logits)

## Train

In [ ]:
EPOCHS = 1
log_interval = 200
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    val_loss.reset_states()
    val_accuracy.reset_states()
    
    val_batches = iter(val_ds)
    
    start_time = time.time()
    for i, (input_ids, input_mask, y) in enumerate(train_ds):
        # training
        #try:
        train_step(input_ids, input_mask, y)
        #except:
        #continue
        # validation
        if i % log_interval == 0:
            x_val, x_mask_val, y_val = next(val_batches)
            val_step(x_val, x_mask_val, y_val)
            elapsed = time.time() - start_time
            print('| epoch {:3d} | [{:5d}/{:5d}] | '
                  'ms/batch {:5.2f} | '
                  'train acc {:5.2f} | val acc {:5.2f} |'
                  'loss {:5.2f} | val loss {:5.2f}'.format(
                    epoch, i, int(len_train/BATCH_SIZE),
                    elapsed * 1000 / log_interval,
                    train_accuracy.result() * 100, val_accuracy.result() * 100, 
                    train_loss.result(),  val_loss.result()))
            start_time = time.time()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


| epoch   0 | [    0/17944] | ms/batch 355.13 | train acc 25.29 | val acc 33.85 |loss 10.75 | val loss  9.94
| epoch   0 | [  200/17944] | ms/batch 808.04 | train acc 61.93 | val acc 57.07 |loss  2.82 | val loss  5.47
| epoch   0 | [  400/17944] | ms/batch 811.30 | train acc 69.21 | val acc 64.57 |loss  2.03 | val loss  3.97
| epoch   0 | [  600/17944] | ms/batch 810.89 | train acc 72.05 | val acc 68.69 |loss  1.73 | val loss  3.23
| epoch   0 | [  800/17944] | ms/batch 811.82 | train acc 73.54 | val acc 71.35 |loss  1.58 | val loss  2.78
| epoch   0 | [ 1000/17944] | ms/batch 811.66 | train acc 74.57 | val acc 71.60 |loss  1.48 | val loss  2.54
| epoch   0 | [ 1200/17944] | ms/batch 811.23 | train acc 75.27 | val acc 72.59 |loss  1.41 | val loss  2.32
| epoch   0 | [ 1400/17944] | ms/batch 811.66 | train acc 75.81 | val acc 73.30 |loss  1.36 | val loss  2.17
| epoch   0 | [ 1600/17944] | ms/batch 812.10 | train acc 76.21 | val acc 74.16 |loss  1.32 | val loss  2.03
| epoch   0 | [ 180

## Evaluate

### Define Rouge Score

In [14]:
!pip install rouge-score

In [15]:
from rouge_score import rouge_scorer
from rouge_score import scoring

class RougeScore:
    
    def __init__(self, score_keys=None)-> None:
        super().__init__()
        if score_keys is None:  
            self.score_keys = ["rouge1", "rouge2", "rougeLsum"]
        
        self.scorer = rouge_scorer.RougeScorer(self.score_keys)
        self.aggregator = scoring.BootstrapAggregator()
        
        
    @staticmethod
    def prepare_summary(summary):
            summary = summary.replace(" . ", " .\n")
            return summary
    
    def __call__(self, target, prediction):
        """Computes rouge score.''
        Args:
        targets: string
        predictions: string
        """

        target = self.prepare_summary(target)
        prediction = self.prepare_summary(prediction)
        
        self.aggregator.add_scores(self.scorer.score(target=target, prediction=prediction))

        return 
    
    def reset_states(self):
        self.rouge_list = []

    def result(self):
        result = self.aggregator.aggregate()
        
        for key in self.score_keys:
            score_text = "%s = %.2f, 95%% confidence [%.2f, %.2f]"%(
                key,
                result[key].mid.fmeasure*100,
                result[key].low.fmeasure*100,
                result[key].high.fmeasure*100
            )
            print(score_text)
        
        return {key: result[key].mid.fmeasure*100 for key in self.score_keys}

In [18]:
test_ds.take(1)

<TakeDataset shapes: ((None, 512), (None, 512), (None, 150)), types: (tf.int32, tf.int32, tf.int32)>

### Compute Rouge Score

In [16]:
predictions = []
rouge_score = RougeScore()
for i, (input_ids, input_mask, y) in enumerate(test_ds):
    summaries = model.generate(input_ids=input_ids, attention_mask=input_mask)

    pred = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summaries]
    real = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in y]
    
    for pred_sent, real_sent in zip(pred, real):
        rouge_score(pred_sent, real_sent)
        predictions.append(str("pred sentence: " + pred_sent + "\n\n real sentence: " + real_sent))
        
    if i > 100:
        # otherwise it will take ages
        break


rouge_score.result()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


rouge1 = 40.12, 95% confidence [39.52, 40.72]
rouge2 = 18.40, 95% confidence [17.83, 19.04]
rougeLsum = 37.11, 95% confidence [36.55, 37.70]


{'rouge1': 40.118984089679145,
 'rouge2': 18.401294761576324,
 'rougeLsum': 37.11376539035381}

In [ ]:
!cp -r '/content/summarisation_model' /content/drive/MyDrive
!cp -r '/content/summarisation_tokeniser' /content/drive/MyDrive

In [ ]:
model.save_pretrained("summarisation_model")
tokenizer.save_pretrained("summarisation_tokeniser")

('summarisation_tokeniser/tokenizer_config.json',
 'summarisation_tokeniser/special_tokens_map.json',
 'summarisation_tokeniser/spiece.model',
 'summarisation_tokeniser/added_tokens.json')

### Predict some Sentences

In [ ]:
for pred in predictions[:10]:
    print("------")
    print(pred)
    print("------")     

------
pred sentence: A cave photographer, john spies, 59, captured the sheer magnificence of the vast, yet intricate, underground wonderland . xe bang fai river caves feature imposing stalagmitemade of mineral deposits . the cave is only able to be safely accessed during the dry season from november to april and during this time the water is clear and deep with a rich green hue . in 2008, an expedition led to the mysterious caves being 

 real sentence: the tham khoun ex cave has 15km of spectacular caves waiting to be explore by kayak . explorers can witness the incredible caverns, lake and even the vibrant forest at the entrance . cave photographer john spies captured the labyrinthine chambers to unfold the mystery .
------
------
pred sentence: houston couple, who have not been named, were arguing in the parking lot of an apartment complex . husband admitted to police that during the heat of the argument he tried to drive off in his green chevrolet pickup truck when his wife grabbe